In [51]:
"""
Generating data

"""

import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import copy
import time
from npl import bootstrap_logreg as bbl
import pickle

import seaborn as sns
import importlib
from npl.evaluate import logreg_ll as lrl



n_iter=5

def gen_toy(N_data, beta, intercept=2, seed=100):
    #def run_HDP_NPL(datasets, method, alpha, beta, params):
    #run_HDP_NPL([d1, d2, d3], method=‘RandomForest’, alpha=0.1, gamma=2., params={num_trees: 3, tree_depth: 5})
    #run_HDP_NPL([d1, d2, d3,d4], method=‘LogisticRegression’, alpha=0.1, gamma=2., params={regularizer:None})
    
    np.random.seed(seed)
    
    K_set=len(N_data)
    X=[]
    
    for ss in range(K_set):
        X.append(np.random.multivariate_normal(beta[ss], [[1,0],[0,1]], N_data[ss]))
    
    Linear_part=[]
    eta=[]
    y=[]
    
    for i in range(K_set):
        Linear_part.append(X[i]@beta[i]+intercept*np.ones(N_data[i]))
        eta.append(1/(1 + np.exp(-Linear_part[i])))
        y.append(np.random.binomial(1,eta[i]))
    
    gamma=[1/b for b in N_data ]
    
    return X, y, gamma


def gen_test_toy(N_test_data, beta, intercept=2, seed=100):
    #def run_HDP_NPL(datasets, method, alpha, beta, params):
    #run_HDP_NPL([d1, d2, d3], method=‘RandomForest’, alpha=0.1, gamma=2., params={num_trees: 3, tree_depth: 5})
    #run_HDP_NPL([d1, d2, d3,d4], method=‘LogisticRegression’, alpha=0.1, gamma=2., params={regularizer:None})
    
    np.random.seed(seed+10101)
    
    K_set=len(N_test_data)
    
    X=[]
    
    for i in range(K_set):
        X.append(np.random.multivariate_normal(beta[i], [[1,0],[0,1]], N_test_data[i]))
        
    Linear_part=[]
    eta=[]
    y=[]
    
    for i in range(K_set):
        Linear_part.append(X[i]@beta[i]+intercept*np.ones(N_test_data[i]))
        eta.append(1/(1 + np.exp(-Linear_part[i])))
        y.append(np.random.binomial(1,eta[i]))
    
    gamma=[1/b for b in N_test_data ]
    
    return X, y, gamma



    

In [52]:
def main(beta, intercept, B_postsamples):
    

    T_trunc = 100
    a=1
    b = 1 #rate of gamma hyperprior
    
    alph_conc=0 # D ~ DP(alpha_conc, H)
    alpha_top_layer=0 # Dk ~ DP(alpha_top_layer, D)
    N_data=[1000,2000,5000]
    D_data=len(beta[0])
    
    K_set=len(beta)
    
    
    for i in range(n_iter):

        seed = 100+i
        
        np.random.seed(seed)
        x, y, gamma = gen_toy(N_data, beta, intercept, seed)
        
        #y,x,alph_conc,gamma,N_data,D_data = load_data(dataset,seed)

        start= time.time()
        #carry out posterior bootstrap
        temp = bbl.bootstrap_logreg(B_postsamples,alph_conc,alpha_top_layer,T_trunc,y,x,N_data,D_data,a,b,gamma)
        end = time.time()
        print ('Time elapsed = {}'.format(end - start))
        
        
        
        beta_bb=[]
        ll_bb=[]
        
        for j in range(K_set):
            beta_b=np.zeros((B_postsamples,K_set))
            ll_b=[]
            for bb in range(B_postsamples):
                beta_b[bb]=temp[j][bb][0]
                ll_b.append(temp[j][bb][1])
            beta_bb.append(beta_b)
            ll_bb.append(ll_b)
        
        

        #convert to dataframe and save
        dict_bb = {'beta': beta_bb, 'll': ll_bb, 'time': end-start}
        par_bb = pd.Series(data = dict_bb)
        print(par_bb['time'])
        
        par_bb.to_pickle('./parameters/par_bb_logreg_c{}_a{}_b{}_gN_pol_B{}_seed{}'.format(alph_conc,a,b,B_postsamples,seed))


if __name__=='__main__':
    main([np.array([-1,1]),np.array([-1,1]),np.array([-1,1])],2,1000)
    #main([np.array([1,5]),np.array([-1,7]),np.array([9,-1])],2,1000)
    
    
    
    
    
    





  0%|          | 0/1000 [00:00<?, ?it/s]



  1%|          | 12/1000 [00:00<00:13, 73.84it/s]



  2%|▏         | 24/1000 [00:01<00:49, 19.91it/s]



  6%|▌         | 60/1000 [00:01<00:34, 27.59it/s]



 16%|█▌        | 156/1000 [00:02<00:21, 38.83it/s]



 30%|███       | 300/1000 [00:02<00:12, 54.56it/s]



 40%|███▉      | 396/1000 [00:02<00:08, 75.20it/s]



 49%|████▉     | 492/1000 [00:02<00:04, 101.68it/s]



 59%|█████▉    | 588/1000 [00:02<00:03, 135.14it/s]



 68%|██████▊   | 684/1000 [00:02<00:01, 175.74it/s]



 78%|███████▊  | 780/1000 [00:02<00:00, 223.91it/s]



 88%|████████▊ | 876/1000 [00:03<00:00, 262.08it/s]



100%|██████████| 1000/1000 [00:03<00:00, 291.60it/s][A




  0%|          | 0/1000 [00:00<?, ?it/s]



  7%|▋         | 72/1000 [00:00<00:01, 617.59it/s]



 17%|█▋        | 168/1000 [00:00<00:01, 642.06it/s]



 36%|███▌      | 360/1000 [00:00<00:00, 685.55it/s]



 46%|████▌     | 456/1000 [00:00<00:00, 681.86it/s]



 55%|█████▌    | 552/1000 [00:00<0

Time elapsed = 9.130327939987183
9.130327939987183






  0%|          | 0/1000 [00:00<?, ?it/s]



  7%|▋         | 72/1000 [00:00<00:01, 634.47it/s]



 17%|█▋        | 168/1000 [00:00<00:01, 606.11it/s]



 36%|███▌      | 360/1000 [00:00<00:00, 647.76it/s]



 46%|████▌     | 456/1000 [00:00<00:00, 590.98it/s]



 55%|█████▌    | 552/1000 [00:00<00:00, 543.36it/s]



 65%|██████▍   | 648/1000 [00:01<00:00, 573.69it/s]



 74%|███████▍  | 744/1000 [00:01<00:00, 595.79it/s]



 84%|████████▍ | 840/1000 [00:01<00:00, 616.84it/s]



100%|██████████| 1000/1000 [00:01<00:00, 646.22it/s][A




  0%|          | 0/1000 [00:00<?, ?it/s]



 12%|█▏        | 120/1000 [00:00<00:00, 947.52it/s]



 26%|██▋       | 264/1000 [00:00<00:00, 885.03it/s]



 36%|███▌      | 360/1000 [00:00<00:00, 693.06it/s]



 46%|████▌     | 456/1000 [00:00<00:00, 605.47it/s]



 55%|█████▌    | 552/1000 [00:00<00:00, 551.69it/s]



 65%|██████▍   | 648/1000 [00:01<00:00, 571.11it/s]



 74%|███████▍  | 744/1000 [00:01<00:00, 584.93it/s]



 84%|████████▍ | 840/1000

Time elapsed = 7.306169033050537
7.306169033050537






  0%|          | 0/1000 [00:00<?, ?it/s]



 12%|█▏        | 120/1000 [00:00<00:01, 851.69it/s]



 26%|██▋       | 264/1000 [00:00<00:00, 897.83it/s]



 36%|███▌      | 360/1000 [00:00<00:00, 790.80it/s]



 46%|████▌     | 456/1000 [00:00<00:00, 744.15it/s]



 55%|█████▌    | 552/1000 [00:00<00:00, 718.49it/s]



 65%|██████▍   | 648/1000 [00:00<00:00, 640.98it/s]



 74%|███████▍  | 744/1000 [00:01<00:00, 654.80it/s]



 84%|████████▍ | 840/1000 [00:01<00:00, 634.57it/s]



100%|██████████| 1000/1000 [00:01<00:00, 713.37it/s][A




  0%|          | 0/1000 [00:00<?, ?it/s]



 12%|█▏        | 120/1000 [00:00<00:00, 1011.10it/s]



 26%|██▋       | 264/1000 [00:00<00:00, 1041.62it/s]



 36%|███▌      | 360/1000 [00:00<00:00, 832.14it/s] 



 46%|████▌     | 456/1000 [00:00<00:00, 768.92it/s]



 55%|█████▌    | 552/1000 [00:00<00:00, 793.81it/s]



 65%|██████▍   | 648/1000 [00:00<00:00, 782.94it/s]



 74%|███████▍  | 744/1000 [00:00<00:00, 667.41it/s]



 84%|████████▍ | 840/

Time elapsed = 6.13362717628479
6.13362717628479






  0%|          | 0/1000 [00:00<?, ?it/s]



 12%|█▏        | 120/1000 [00:00<00:00, 989.18it/s]



 26%|██▋       | 264/1000 [00:00<00:00, 1006.57it/s]



 36%|███▌      | 360/1000 [00:00<00:00, 897.52it/s] 



 46%|████▌     | 456/1000 [00:00<00:00, 830.06it/s]



 55%|█████▌    | 552/1000 [00:00<00:00, 792.76it/s]



 65%|██████▍   | 648/1000 [00:00<00:00, 778.53it/s]



 74%|███████▍  | 744/1000 [00:00<00:00, 739.58it/s]



 84%|████████▍ | 840/1000 [00:01<00:00, 669.35it/s]



100%|██████████| 1000/1000 [00:01<00:00, 776.95it/s][A




  0%|          | 0/1000 [00:00<?, ?it/s]



 12%|█▏        | 120/1000 [00:00<00:00, 1101.35it/s]



 26%|██▋       | 264/1000 [00:00<00:00, 1097.69it/s]



 36%|███▌      | 360/1000 [00:00<00:00, 960.03it/s] 



 55%|█████▌    | 552/1000 [00:00<00:00, 945.08it/s]



 74%|███████▍  | 744/1000 [00:00<00:00, 770.83it/s]



100%|██████████| 1000/1000 [00:01<00:00, 777.68it/s][A




  0%|          | 0/1000 [00:00<?, ?it/s]



 12%|█▏        | 120/1000 

Time elapsed = 6.386575222015381
6.386575222015381






  0%|          | 0/1000 [00:00<?, ?it/s]



  7%|▋         | 72/1000 [00:00<00:01, 564.85it/s]



 17%|█▋        | 168/1000 [00:00<00:01, 609.62it/s]



 36%|███▌      | 360/1000 [00:00<00:00, 670.68it/s]



 46%|████▌     | 456/1000 [00:00<00:00, 599.42it/s]



 55%|█████▌    | 552/1000 [00:00<00:00, 498.09it/s]



 65%|██████▍   | 648/1000 [00:01<00:00, 481.50it/s]



 74%|███████▍  | 744/1000 [00:01<00:00, 499.54it/s]



 84%|████████▍ | 840/1000 [00:01<00:00, 550.17it/s]



100%|██████████| 1000/1000 [00:01<00:00, 605.39it/s][A




  0%|          | 0/1000 [00:00<?, ?it/s]



  5%|▍         | 48/1000 [00:00<00:03, 277.65it/s]



 17%|█▋        | 168/1000 [00:00<00:02, 344.67it/s]



 36%|███▌      | 360/1000 [00:00<00:01, 387.87it/s]



 46%|████▌     | 456/1000 [00:00<00:01, 398.20it/s]



 55%|█████▌    | 552/1000 [00:01<00:01, 379.85it/s]



 65%|██████▍   | 648/1000 [00:01<00:00, 403.95it/s]



 74%|███████▍  | 744/1000 [00:01<00:00, 441.53it/s]



 84%|████████▍ | 840/1000 

Time elapsed = 7.349469900131226
7.349469900131226


In [53]:
X_test, y_test, _=gen_test_toy([1000,1000,1000], [np.array([0,0]),np.array([-1,1]),np.array([1,-1])],2,12121)

In [54]:
def eval(x_test, y_test, N_test):
    #par=dataset
    #Load test data
    K_set=len(y_test)
    
    
    lppd = np.zeros([n_iter,K_set])
    mse = np.zeros([n_iter,K_set])
    predcor = np.zeros([n_iter,K_set])
    time = np.zeros([n_iter,K_set])
    card = np.zeros([n_iter,K_set])
    
    
    D,c,eps = 2, 0, 1e-1
    
    
    

    for i in range(n_iter):
        par = pd.read_pickle('./parameters/par_bb_logreg_c{}_a{}_b{}_gN_pol_B1000_seed{}'.format(0,1,1,100+i))
        #Run through each seed to calculate predictive performance/times
        #seed = 100+i
        #y_test,x_test,N_test,D,c,pref,eps = load_data(dataset,seed)
        BETA=[]
        ALPHA=[]
        time[i]=par['time']
        for j in range(K_set):
            BETA.append( par['beta'][j][:,0:2])
            ALPHA.append(par['beta'][j][:,2])
            
            lppd[i,j]=lrl.lppd(y_test[j],x_test[j],BETA[j],ALPHA[j])
            mse[i,j]=lrl.MSE(y_test[j],x_test[j],BETA[j],ALPHA[j])
            predcor[i,j]= lrl.predcorrect(y_test[j],x_test[j],BETA[j],ALPHA[j])
            mean_beta = np.mean(BETA[j],axis = 0)
            card[i,j] = lrl.checkcard([mean_beta],eps)
        
        
    #dict = {'lppd_1':lppd[0]/N_test[0], 'mse_1':mse[0], 'predcor_1': 100*predcor[0], 'card_1': ((D-card[0])/D)*100, 'time':time }

    for j in range(K_set):
        print('FOR DATASET')
        print(j)
        
        print('lppd')
        print(np.mean(lppd[:,j]/N_test[j]))
        print(np.std(lppd[:,j]/N_test[j]))
        
        #import pdb
        #pdb.set_trace()

        print('mse')
        print(np.mean(mse[:,j]))
        print(np.std(mse[:,j]))

        print('pa')
        print(np.mean(100*predcor[:,j]))
        print(np.std(100*predcor[:,j]))

    
        print('time')
        print(np.mean(time))
        print(np.std(time))
        
        print("\n")
        print("\n")
        
        


#Run for 3 datasets from paper
def main():
    eval(X_test,y_test,[1000,1000,1000])

if __name__=='__main__':
    main()

FOR DATASET
0
lppd
-0.4646786828113344
0.03622177235858744
mse
0.13620489835314536
0.011679005470384616
pa
83.26
1.7579533554676536
time
7.261233854293823
1.0526007804061877




FOR DATASET
1
lppd
-0.1325502102866053
0.0010226552585402247
mse
0.03381235404125658
0.00029862623614619556
pa
95.99999999999999
0.1414213562373115
time
7.261233854293823
1.0526007804061877




FOR DATASET
2
lppd
-0.9105903865010376
0.07570394990393282
mse
0.3181083916144147
0.026192267010158497
pa
50.56
4.277662913320776
time
7.261233854293823
1.0526007804061877






In [ ]:
par = pd.read_pickle('./parameters/par_bb_logreg_c{}_a{}_b{}_gN_pol_B1000_seed{}'.format(0,1,1,100))

In [ ]:
beta_2=par["beta"]

In [ ]:
np.mean(beta_2[2],axis=0)

In [ ]:
plt.hist(beta_2[2][:,0:2].flatten())
plt.show()

In [ ]:
import sklearn
from sklearn.linear_model import LogisticRegression

In [ ]:

N_data=[100,200,500]
X, y, gamma,eta = gen_toy(N_data, [np.array([-1,1]),np.array([-1,1]),np.array([-1,1])],2, 100)

In [ ]:
[np.sum(yy) for yy in y]

In [ ]:
plt.hist(eta)
plt.show()

In [ ]:
clf = LogisticRegression(random_state=0).fit(X[1], y[1])
clf.intercept_

In [ ]:
clf.coef_

In [ ]:
model=sklearn.linear_model.LogisticRegression(penalty='l2', *, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)[source]





## Adult

In [63]:
import numpy as np
import pandas as pd
from scipy.io import arff
import random
import pickle
from sklearn.model_selection import train_test_split

def adult_load(seed):
    np.random.seed(seed)

    #import
    ad_train = pd.read_csv('./data/adult.data',header = None)
    #import pdb
    #pdb.set_trace()
    ad_test = pd.read_csv('./data/adult.test', header = None)

    #convert missing data to Nans
    ad_train[ad_train == ' ?']= np.nan
    ad_test[ad_test == ' ?']= np.nan

    #drop missing categorical data
    ad_train.dropna(axis = 0,inplace = True)
    ad_test.dropna(axis = 0,inplace = True)

    #separate covariates from classes
    N_train = np.shape(ad_train)[0]
    y_train = np.zeros(N_train)
    y_train[ad_train.iloc[:,14] == ' >50K']=1
    x_train = ad_train.iloc[:,0:14]

    N_test = np.shape(ad_test)[0]
    y_test = np.zeros(N_test)
    y_test[ad_test.iloc[:,14] == ' >50K.']=1
    x_test = ad_test.iloc[:,0:14]

    #setup dummy
    x_train =pd.get_dummies(x_train,drop_first = True,columns = [1,3,5,6,7,8,9,13])
    x_test = pd.get_dummies(x_test,drop_first = True,columns = [1,3,5,6,7,8,9,13])    

    #fix dummy difference
    missing_cols = set( x_train.columns ) - set( x_test.columns)
    for c in missing_cols:
        x_test[c] = 0
    x_test = x_test[x_train.columns]

    D = np.shape(x_train)[1]
    colnames = list(x_train.columns.values)

    #concatenate and resplit
    x = np.concatenate((x_train,x_test),axis = 0)
    y = np.concatenate((y_train,y_test))
    x_train,x_test, y_train,y_test = train_test_split(x,y,test_size = 0.2, stratify = y)
    x_train = pd.DataFrame(x_train,columns = colnames)
    x_test = pd.DataFrame(x_test, columns = colnames)
    y_train = pd.DataFrame(y_train)
    y_test = pd.DataFrame(y_test)
    
    #import pdb
    #pdb.set_trace()

    N_train = np.shape(x_train)[0]
    N_test = np.shape(x_test)[0]

    #normalize by mean and std for non dummy variables
    mean_train = x_train.mean(axis = 0)
    std_train = x_train.std(axis = 0)
    x_train[[0,2,4,10,11,12]] -= mean_train
    x_train[[0,2,4,10,11,12]] /= std_train

    mean_test = x_test[[0,2,4,10,11,12]].mean(axis = 0)
    std_test = x_test[[0,2,4,10,11,12]].std(axis = 0)
    x_test[[0,2,4,10,11,12]] -= mean_test
    x_test[[0,2,4,10,11,12]] /= std_test

    #convert binarys to uint8 to save space
    y_train = y_train.astype('uint8')    
    y_test = y_test.astype('uint8')

    colnames2 = set(colnames) - set([0,2,4,10,11,12])
    x_train[list(colnames2)] = x_train[list(colnames2)].astype('uint8') 
    x_test[list(colnames2)] = x_test[list(colnames2)].astype('uint8') 

    #Put into dictionary and save
    ad_data_train= {'y': y_train, 'x': x_train, 'N':N_train,'D':D}
    ad_data_test= {'y': y_test, 'x': x_test, 'N':N_test,'D':D}

    with open('./data/ad_train_seed{}'.format(seed), 'wb') as handle:
        pickle.dump(ad_data_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open('./data/ad_test_seed{}'.format(seed), 'wb') as handle:
        pickle.dump(ad_data_test, handle, protocol=pickle.HIGHEST_PROTOCOL)



for i in range(30):
    adult_load(100+i)
  

/Users/cuifuheng/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/cuifuheng/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/cuifuheng/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/cuifuheng/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [65]:
"""
main script for running NPL

"""

import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import copy
import time
from npl import bootstrap_logreg as bbl
import pickle
n_iter=1

def load_data(seed):
    #load adult
    if True:
        with open('./data/ad_train_seed{}'.format(seed), 'rb') as handle:
            ad_train = pickle.load(handle)

        #Move into vectors
        y = np.uint8(ad_train['y'])[:,0]
        x = ad_train['x'].values
        D_data = ad_train['D']
        N_data = ad_train['N']
  

        #prior and loss settings from paper
        alph_conc = 0 
        gamma = 1/N_data


    return y,x,alph_conc,gamma,N_data,D_data



def main(B_postsamples):
    #same parameters between datasets
    T_trunc = 100
    a=1
    b = 1 #rate of gamma hyperprior
    
    
    alpha_top_layer=0
    alph_conc=0
    
    
    K_set=2
    for i in range(n_iter):

        seed = 100+i
        np.random.seed(seed)
        y,x,_,_,N_data,D_data = load_data(seed)
        
        x_m=x[x[:,55]==1,:]
        x_f=x[x[:,55]==0,:]
        
        y_m=y[x[:,55]==1]
        y_f=y[x[:,55]==0]
        
        N_data=[int(sum(x[:,55]==1)),int(sum(x[:,55]==0))]
        y=[y_m,y_f]
        x=[x_m,x_f]
        
        gamma=[1/N_data[0],1/N_data[1]]
    

        start= time.time()
        #carry out posterior bootstrap
        temp = bbl.bootstrap_logreg(B_postsamples,alph_conc,alpha_top_layer,T_trunc,y,x,N_data,D_data,a,b,gamma)
        end = time.time()
        print ('Time elapsed = {}'.format(end - start))
        
        
        
        beta_bb=[]
        ll_bb=[]
        
        for j in range(K_set):
            beta_b=np.zeros((B_postsamples,D_data+1))
            ll_b=[]
            for bb in range(B_postsamples):
                beta_b[bb]=temp[j][bb][0]
                ll_b.append(temp[j][bb][1])
            beta_bb.append(beta_b)
            ll_bb.append(ll_b)
        
        

        #convert to dataframe and save
        dict_bb = {'beta': beta_bb, 'll': ll_bb, 'time': end-start}
        par_bb = pd.Series(data = dict_bb)
        print(par_bb['time'])
        
        
        

        
        if True:
            par_bb.to_pickle('./parameters/par_ad_bb_logreg_c{}_a{}_b{}_gN_ad_B{}_seed{}'.format(alph_conc,a,b,B_postsamples,seed))

        

if __name__=='__main__':
    main(20)











  0%|          | 0/20 [00:00<?, ?it/s]




100%|██████████| 20/20 [00:00<00:00, 107.84it/s][A





100%|██████████| 20/20 [00:00<00:00, 6968.44it/s]A


Time elapsed = 209.41177582740784
209.41177582740784


In [ ]:
## 55 male =1 felmale =0